In [1]:
import os
import zipfile
import shutil
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import unicodedata

In [7]:
# URL del CSV con los enlaces de descarga
csv_url = 'https://docs.google.com/spreadsheets/d/1azJrrovCMYBeRasAOwh-87nVSxQPdjRs2AfBoKy97-0/export?format=csv'
output_directory = 'drive'
temp_directory = 'temp_dataset'

In [8]:
# Crear los directorios necesarios
os.makedirs(output_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)

In [10]:
# Configuración del navegador para la descarga automática
chrome_options = webdriver.ChromeOptions()
prefs = {
    'download.default_directory': os.path.abspath(output_directory),
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [11]:
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 60)

In [12]:
# Descargar el archivo CSV con los enlaces de descarga
response = requests.get(csv_url)
url_drives = []
if response.status_code == 200:
    drives = response.content.decode('utf-8').split('\n')
    for i in range(1, len(drives)):
        drives[i] = drives[i].replace('no con jpeg y txt', '')
        drives[i] = drives[i].replace('"', '')
        columns = drives[i].split(',')
        link = columns[-1].strip()
        url_drives.append(link)
else:
    print(f"Error al acceder a la hoja de cálculo: {response.status_code}")


In [ ]:
# Descargar los archivos desde Google Drive
for folder_url in url_drives:
    driver.get(folder_url)
    time.sleep(10)  # Aumentar el tiempo de espera para asegurar que la página se cargue completamente

    try:
        # Intenta encontrar el botón de descarga
        download_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "Descargar") or contains(text(), "Download")]'))
        )
        download_button.click()
        time.sleep(20)  # Esperar a que la descarga comience
        print(f"Descarga iniciada desde la carpeta: {folder_url}")
    except Exception as e:
        print(f"Error al intentar descargar archivos de la carpeta {folder_url}: {e}")

driver.quit()
print("Descarga completada.")

In [13]:
# Directorios finales para las imágenes y las etiquetas
images_directory = 'dataset_tmp/images'
labels_directory = 'dataset_tmp/labels'
extensiones_imagenes = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

# Crear directorios de imágenes y etiquetas si no existen
os.makedirs(images_directory, exist_ok=True)
os.makedirs(labels_directory, exist_ok=True)

In [2]:
# Función para normalizar nombres de archivo
def normalize_filename(filename):
    filename = filename.lower()
    filename = unicodedata.normalize('NFKD', filename).encode('ascii', 'ignore').decode('ascii')
    filename = filename.replace(' ', '-').replace('_', '')
    return filename


In [4]:
# Función para copiar pares de archivos a los directorios correspondientes
def copiar_pares(pares, img_dest, lbl_dest):
    for imagen_path, txt_path in pares:
        imagen_name = normalize_filename(os.path.basename(imagen_path))
        txt_name = normalize_filename(os.path.basename(txt_path))

        shutil.copy(imagen_path, os.path.join(img_dest, imagen_name))
        shutil.copy(txt_path, os.path.join(lbl_dest, txt_name))

        os.remove(imagen_path)
        os.remove(txt_path)

In [16]:
# Procesar los archivos descargados
if os.path.exists(output_directory):
    for file in os.listdir(output_directory):
        if file.endswith('.zip'):
            file_path = os.path.join(output_directory, file)
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(temp_directory)
                print(f"Unzipped '{file}' in '{temp_directory}'")

    for item in os.listdir(temp_directory):
        item_path = os.path.join(temp_directory, item)
        if os.path.isdir(item_path):
            pares_imagen_txt = []

            for archivo in os.listdir(item_path):
                if os.path.splitext(archivo)[1].lower() in extensiones_imagenes:
                    archivo_txt = os.path.splitext(archivo)[0] + '.txt'
                    if archivo_txt in os.listdir(item_path):
                        pares_imagen_txt.append(
                            (os.path.join(item_path, archivo), os.path.join(item_path, archivo_txt)))
                    else:
                        os.remove(os.path.join(item_path, archivo))
                        print(f"Imagen '{archivo}' eliminada porque no tiene un archivo .txt correspondiente.")

            copiar_pares(pares_imagen_txt, images_directory, labels_directory)
            print(f"Imágenes y etiquetas movidas desde '{item}'")

    # Limpiar la carpeta de descarga
    shutil.rmtree(output_directory)
    print(f"Carpeta '{output_directory}' eliminada.")
else:
    print(f"El directorio '{output_directory}' no existe.")

Unzipped 'imagenes-20240710T141637Z-001.zip' in 'temp_dataset'
Unzipped 'dataset coputer vision-20240710T142236Z-001.zip' in 'temp_dataset'
Unzipped 'truco_simon_revello-20240710T143831Z-001.zip' in 'temp_dataset'
Unzipped 'Images_with_labels-20240710T070751Z-001.zip' in 'temp_dataset'
Unzipped 'Imágenes Dataset Leandro Salvañá-20240710T142147Z-001.zip' in 'temp_dataset'
Unzipped 'fotos_tp_cv-20240710T051846Z-001.zip' in 'temp_dataset'
Unzipped 'data_truco-20240710T143141Z-001.zip' in 'temp_dataset'
Unzipped 'TUIA-VC-W05576-20240710T143441Z-001.zip' in 'temp_dataset'
Unzipped 'Fotos cartas -20240710T143232Z-001.zip' in 'temp_dataset'
Unzipped 'Imágenes de cartas - TP Final Visión por Computadora-20240710T070526Z-001.zip' in 'temp_dataset'
Unzipped 'YOLODataset-20240710T142356Z-001.zip' in 'temp_dataset'
Unzipped 'CV - Truco-20240710T143041Z-001.zip' in 'temp_dataset'
Unzipped 'DATASET-20240710T141245Z-001.zip' in 'temp_dataset'
Unzipped 'eugenio_lopez_dataset_truco-20240710T142604Z-001

In [5]:
import os
import shutil
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [18]:
# Directorios de entrada y salida
temp_images_directory = 'dataset_tmp/images'
temp_labels_directory = 'dataset_tmp/labels'
final_images_directory = 'dataset_tmp/images'  # Directorio final para imágenes válidas
final_labels_directory = 'dataset_tmp/labels'  # Directorio final para etiquetas válidas
label_errors_directory = 'dataset/label_errors'
os.makedirs(final_images_directory, exist_ok=True)
os.makedirs(final_labels_directory, exist_ok=True)
os.makedirs(label_errors_directory, exist_ok=True)


In [19]:
# Función para verificar y unificar anotaciones en formato YOLOv5
def validar_y_unificar_anotaciones(images_dir, labels_dir, final_images_dir, final_labels_dir, label_errors_dir):
    extensiones_imagenes = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    archivos_procesados = 0
    archivos_con_errores = 0

    for label_file in tqdm(os.listdir(labels_dir)):
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_dir, label_file)
            image_path = None

            for ext in extensiones_imagenes:
                potential_image_path = os.path.join(images_dir, os.path.splitext(label_file)[0] + ext)
                if os.path.exists(potential_image_path):
                    image_path = potential_image_path
                    break

            if image_path is None:
                shutil.move(label_path, os.path.join(label_errors_dir, os.path.basename(label_path)))
                archivos_con_errores += 1
                continue

            # Leer imagen
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            # Leer anotaciones
            with open(label_path, 'r') as f:
                lines = f.readlines()

            valid_lines = []
            error_found = False
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    id_clase, x_centro, y_centro, ancho, alto = map(float, parts)
                elif len(parts) == 9:
                    id_clase, x_centro, y_centro, ancho, alto = map(float, parts[:5])
                else:
                    error_found = True

                # Verificar límites y convertir a formato YOLOv5
                if not (0 <= x_centro <= 1 and 0 <= y_centro <= 1 and 0 <= ancho <= 1 and 0 <= alto <= 1):
                    error_found = True

                # Verificar que la etiqueta esté en el rango 0-48
                if not (0 <= id_clase <= 48):
                    error_found = True

                if error_found:
                    break
                else:
                    valid_lines.append(f"{int(id_clase)} {x_centro} {y_centro} {ancho} {alto}\n")

            if error_found:
                archivos_con_errores += 1
                shutil.move(label_path, os.path.join(label_errors_dir, os.path.basename(label_path)))
                os.remove(image_path)
            else:
                # Escribir anotaciones válidas y mover archivos
                with open(os.path.join(final_labels_dir, os.path.basename(label_path)), 'w') as f:
                    f.writelines(valid_lines)
                archivos_procesados += 1

    print(f"Archivos procesados: {archivos_procesados}")
    print(f"Archivos con errores: {archivos_con_errores}")

validar_y_unificar_anotaciones(temp_images_directory, temp_labels_directory, final_images_directory, final_labels_directory, label_errors_directory)

100%|██████████| 1513/1513 [02:10<00:00, 11.59it/s]

Archivos procesados: 1434
Archivos con errores: 79


In [ ]:
validar_y_unificar_anotaciones(temp_images_directory, temp_labels_directory, final_images_directory, final_labels_directory, label_errors_directory)

In [20]:
# División del dataset en entrenamiento, validación y prueba
image_files = [f for f in os.listdir(final_images_directory) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)

In [21]:
# Crear directorios para los splits
split_dirs = {
    'train': {'images': 'dataset/train/images', 'labels': 'dataset/train/labels'},
    'val': {'images': 'dataset/val/images', 'labels': 'dataset/val/labels'},
    'test': {'images': 'dataset/test/images', 'labels': 'dataset/test/labels'}
}
for split in split_dirs:
    os.makedirs(split_dirs[split]['images'], exist_ok=True)
    os.makedirs(split_dirs[split]['labels'], exist_ok=True)

In [22]:
# Función para mover archivos a sus directorios correspondientes
def mover_archivos(files, split):
    for file in files:
        shutil.copy(os.path.join(final_images_directory, file), split_dirs[split]['images'])
        shutil.copy(os.path.join(final_labels_directory, os.path.splitext(file)[0] + '.txt'), split_dirs[split]['labels'])

mover_archivos(train_files, 'train')
mover_archivos(val_files, 'val')
mover_archivos(test_files, 'test')

# Reporte de la cantidad de imágenes en cada split
print(f"Cantidad de imágenes en el conjunto de entrenamiento: {len(train_files)}")
print(f"Cantidad de imágenes en el conjunto de validación: {len(val_files)}")
print(f"Cantidad de imágenes en el conjunto de prueba: {len(test_files)}")

Cantidad de imágenes en el conjunto de entrenamiento: 860
Cantidad de imágenes en el conjunto de validación: 287
Cantidad de imágenes en el conjunto de prueba: 287
